In [1]:
import numpy as np
from pyscf import gto, scf, df, ao2mo

In [70]:
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='STO-3G', charge = 1)
#mol = gto.M(atom='He 0 0 0; H 0 0 1', basis='pc-3', charge = 1)
#mol = gto.M(atom='Be 0 0 0', basis='6-31G', charge = 0)
mol = gto.M(atom='H 0 0 0; H 0 0 0.740848149', basis='STO-3G', charge = 0)
#mol = gto.M(atom="O 0.00000 0.00000 0.11779; H 0.00000 0.75545 -0.47116; H 0.00000 -0.75545 -0.47116", basis = "sto-3g")
#mol = gto.M(atom="O 0.00000 -0.143225816552  0.00000 ; H 1.638036840407   1.136548822547  -0.000000000000; H -1.638036840407   1.136548822547  -0.000000000000", basis = "sto-3g")
#mol = gto.M(atom="O 0.00000 -0.075791807925  0.00000 ; H 0.8668114 0.6014355  -0.000000000000; H -0.8668114 0.6014355  -0.000000000000", basis = "sto-3g")

In [71]:
#calculate nuclear repulsion energy
tot_nuc_energy=0
for i in range(0,len(mol.atom_coords())-1):
    for j in range(0,len(mol.atom_coords())):
        if j > i:
            print("i",i,"-- j",j)
            x=mol.atom_coord(i)-mol.atom_coord(j)
           # print("x",x)
            norm=np.linalg.norm(x)
           # print("norm",norm)
            nuc_energy=mol.atom_charge(i)*mol.atom_charge(j)/norm
            print("nuc_energy",nuc_energy)     
            tot_nuc_energy+=nuc_energy
print("tot_nuc_energy",tot_nuc_energy)

i 0 -- j 1
nuc_energy 0.7142856624995092
tot_nuc_energy 0.7142856624995092


In [75]:
#get integrals and calculate core Hamiltonian

overlap_integrals = mol.intor('int1e_ovlp_sph')
#print("overlap_integrals")
#print(overlap_integrals)
kinetic_energy_integrals = mol.intor('int1e_kin_sph')
#print("kinetic_energy_integrals")
#print(kinetic_energy_integrals)
nuclear_attraction_integrals = mol.intor('int1e_nuc_sph')
#print("nuclear_attraction_integrals")
#print(nuclear_attraction_integrals)
two_electron_integrals = mol.intor('int2e_sph')
#print("two_electron_integrals")
#print(two_electron_integrals)
H_core = kinetic_energy_integrals + nuclear_attraction_integrals


In [76]:
#get integrals and calculate core Hamiltonian

#overlap_integrals = mol.intor('int1e_ovlp_spinor')
#print("overlap_integrals")
#print(overlap_integrals)
#kinetic_energy_integrals = mol.intor('int1e_kin_spinor')
#print("kinetic_energy_integrals")
#print(kinetic_energy_integrals)
#nuclear_attraction_integrals = mol.intor('int1e_nuc_spinor')
#print("nuclear_attraction_integrals")
#print(nuclear_attraction_integrals)
#two_electron_integrals = mol.intor('int2e_spinor')
#print("two_electron_integrals")
#print(two_electron_integrals)
#H_core = kinetic_energy_integrals + nuclear_attraction_integrals


In [77]:
#diagonalize overlap matrix
overlap_diag_procedure = np.linalg.eigh(overlap_integrals)
#get U and S^-0.5
U = overlap_diag_procedure[1]
s_inverse_sqrt = np.diag(1/overlap_diag_procedure[0])**0.5
#get transformation matrix X
X = np.matmul(U,np.matmul(s_inverse_sqrt,np.matrix.transpose(U)))

In [78]:
#set guess for P matrix, default core hamiltonian
P = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

In [79]:
#start iterations
counter = 0
maxiter = 100
energy_difference=10000
energy_last_iter=10000
energy=0

for counter in range(0,maxiter):
    G = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))

    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    G[i,j]+=P[k,l]*(two_electron_integrals[i,j,k,l]-0.5*two_electron_integrals[i,l,k,j])

    F = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    F = H_core + G
#    print(F)
    
    if counter >= 1:
        energy = 0
        for i in range(0,overlap_integrals.shape[0]):
            for j in range(0,overlap_integrals.shape[0]):
                energy += 0.5*(P[i,j]*(H_core[i,j]+F[i,j]))
        energy_difference = abs(energy-energy_last_iter)
        print('iteration {:<9}'.format(counter),"energy = {:<25}".format(energy), "difference = {:<23}".format(energy_difference))
        energy_last_iter = energy
    if energy_difference < 10**-8:
        print("energy converged")
        SCF_final_energy = energy + tot_nuc_energy
        print("final SCF energy including nuclear repulsion:",SCF_final_energy)
        break
    counter+=1    

    F_prime = np.matmul(np.matrix.transpose(X),np.matmul(F,X))
    F_diag_procedure = np.linalg.eigh(F_prime)
    
    C = np.matmul(X,F_diag_procedure[1])
    
    P_new = np.zeros((overlap_integrals.shape[0], overlap_integrals.shape[0]))
    
    for i in range(0,overlap_integrals.shape[0]):
        for j in range(0,overlap_integrals.shape[1]):    
            for a in range(0,mol.nelectron//2):
                P_new[i,j]+=2*C[i,a]*C[j,a]

    P = P_new

iteration 1         energy = -1.830999984673943        difference = 10001.830999984673     
iteration 2         energy = -1.8309999846739433       difference = 2.220446049250313e-16  
energy converged
final SCF energy including nuclear repulsion: -1.1167143221744342


In [80]:
F_diag_procedure = np.linalg.eigh(F_prime)
F_diag_procedure

(array([-0.57820296,  0.67026771]), array([[-0.70710678, -0.70710678],
        [-0.70710678,  0.70710678]]))

In [81]:
#for i in range(0,2):
#    print(C[i,0])

In [82]:
#np.set_printoptions(precision=3)
#np.dot(C.T,np.dot(F,C))

In [83]:
#F

In [84]:
def integraltransformation(I, C):
    # N^5 scaling
    K = np.einsum('pi,ijkl->pjkl', C, I)
    K = np.einsum('qj,pjkl->pqkl', C, K)
    K = np.einsum('rk,pqkl->pqrl', C, K)
    K = np.einsum('sl,pqrl->pqrs', C, K)
    return K

In [85]:
C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))
C_ao_mo[:,:]=C[:,:]

#for i in range(0,C_ao_mo.shape[0]):
#    for p in range(0,C_ao_mo.shape[1]):
#        C_ao_mo[i,p]=C[i,p]
        
mo_integrals = integraltransformation(two_electron_integrals,C_ao_mo)
mo_integrals

array([[[[ 4.85790562, -2.28706836],
         [-2.28706836,  1.71725587]],

        [[-2.28706836,  1.23494981],
         [ 1.23494981, -0.90923939]]],


       [[[-2.28706836,  1.23494981],
         [ 1.23494981, -0.90923939]],

        [[ 1.71725587, -0.90923939],
         [-0.90923939,  0.6783485 ]]]])

In [86]:
#mo_integrals_spin = np.zeros((len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals))*2)
#for p in range(0,len(mo_integrals_spin)):
#    for r in range(0,len(mo_integrals_spin)):
#        print("pr:", p,r)
#        if (p%2==r%2):
#            for q in range(0,len(mo_integrals_spin)):
#                for s in range(0,len(mo_integrals_spin)):
#                    if (q%2==s%2):
#                        print("qs:", q,s)
#                        mo_integrals_spin[p,q,r,s]=mo_integrals[p//2,q//2,r//2,s//2]
#mo_integrals_spin

In [87]:
mo_integrals_spin = np.zeros((len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2,len(mo_integrals)*2))
for p in range(1,len(mo_integrals_spin)+1):
    for r in range(1,len(mo_integrals_spin)+1):
        if (p%2==r%2):
            #print("pr:", p,r)
            for q in range(1,len(mo_integrals_spin)+1):
                for s in range(1,len(mo_integrals_spin)+1):
                    if (q%2==s%2):
                        #print("qs:", q,s)
                        mo_integrals_spin[p-1,q-1,r-1,s-1]=mo_integrals[(p+1)//2-1,(q+1)//2-1,(r+1)//2-1,(s+1)//2-1]
mo_integrals_spin

array([[[[ 4.85790562,  0.        , -2.28706836,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [-2.28706836,  0.        ,  1.71725587,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[ 0.        ,  4.85790562,  0.        , -2.28706836],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        , -2.28706836,  0.        ,  1.71725587],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[-2.28706836,  0.        ,  1.23494981,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 1.23494981,  0.        , -0.90923939,  0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ]],

        [[ 0.        , -2.28706836,  0.        ,  1.23494981],
         [ 0.        ,  0.        ,  0.        ,  0.        ],
         [ 0.        ,  1.23494981,  0.        , -0.90923939],
         [ 0.        ,  0.        ,  0.        , 

In [88]:
B = np.zeros((mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron),mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron)))

#mf = scf.RHF(mol).run()
#mo_ints = ao2mo.kernel(mol, mf.mo_coeff)
#mo_ints = ao2mo.restore(1, mo_ints, mol.nao_nr())
#mo_integrals = mo_ints

arindex = -1
for a in range(0,mol.nelectron):
    for r in range(mol.nelectron,overlap_integrals.shape[0]*2):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron):
            for s in range(mol.nelectron,overlap_integrals.shape[0]*2):
                bsindex += 1
                B[arindex,bsindex]+=mo_integrals_spin[r,s,a,b]-mo_integrals_spin[r,s,b,a]
                #print(arindex,bsindex)
               # lol[a,b] = 1
               # lol[r,s] = 2
                #print(r,s,a,b,mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a],mo_integrals[r,s,a,b],mo_integrals[r,s,b,a])
#assert np.allclose(B, np.zeros(B.shape))
print(B)

[[ 0.          0.          0.          1.71725587]
 [ 0.          0.         -1.71725587  0.        ]
 [ 0.         -1.71725587  0.          0.        ]
 [ 1.71725587  0.          0.          0.        ]]


In [94]:
#print(len(mo_integrals_spin[0])
#,len(mo_integrals_spin[1])
#,len(mo_integrals_spin[2])
#,len(mo_integrals_spin[3]))

In [99]:
eigenenergies=np.zeros((len(F_diag_procedure[0])*2))
for i in range(0,len(eigenenergies)):
    eigenenergies[i]+=F_diag_procedure[0][i//2]
print(eigenenergies)

A = np.zeros((mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron),mol.nelectron*(overlap_integrals.shape[0]*2-mol.nelectron)))

arindex = -1
for a in range(0,mol.nelectron):
    for r in range(mol.nelectron,overlap_integrals.shape[0]*2):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron):
            for s in range(mol.nelectron,overlap_integrals.shape[0]*2):
                bsindex += 1
                A[arindex,bsindex]+=mo_integrals_spin[r,b,a,s]-mo_integrals_spin[r,b,s,a]
                if a==b:
                    A[arindex,bsindex] += eigenenergies[r]
                if r==s:
                    A[arindex,bsindex] += -eigenenergies[a]
                    
print(A)

[-0.57820296 -0.57820296  0.67026771  0.67026771]
[[1.24847067 0.67026771 0.57820296 1.23494981]
 [0.67026771 0.01352086 0.         0.57820296]
 [0.57820296 0.         0.01352086 0.67026771]
 [1.23494981 0.57820296 0.67026771 1.24847067]]


In [100]:
mat = np.matmul((A+B),(A-B)) 

In [101]:
mat

array([[ 0.91838977,  3.54576864,  3.85947654,  3.85869877],
       [-0.42592104, -2.16520745,  0.77510154, -0.74460812],
       [-0.74460812,  0.77510154, -2.16520745, -0.42592104],
       [ 3.85869877,  3.85947654,  3.54576864,  0.91838977]])

In [102]:
mat_diag_procedure = np.linalg.eigh(mat)
print("hartree", mat_diag_procedure[0]**(0.5))
print("eV", mat_diag_procedure[0]**(0.5)*27.2116)

hartree [       nan        nan 0.95016196 2.53818779]
eV [        nan         nan 25.85542728 69.06815096]


/home/peter/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  
/home/peter/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in sqrt
  This is separate from the ipykernel package so we can avoid doing imports until


In [65]:
integrals_ijks = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_ijrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_iqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
integrals_pqrs = np.zeros((overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0],overlap_integrals.shape[0]))
C_ao_mo = np.zeros((F_diag_procedure[1].shape[0],F_diag_procedure[1].shape[1]))


for i in range(0,C_ao_mo.shape[0]):
    for p in range(0,C_ao_mo.shape[1]):
        C_ao_mo[i,p]=C[i,p]


for s in range(0,integrals_ijks.shape[0]):
    for i in range(0,two_electron_integrals.shape[0]):
        for j in range(0,two_electron_integrals.shape[1]):
            for k in range(0,two_electron_integrals.shape[2]):
                for l in range(0,two_electron_integrals.shape[3]):
                    integrals_ijks[i,j,k,s]+=C_ao_mo[l,s]*two_electron_integrals[i,j,k,l]
    
for s in range(0,integrals_ijrs.shape[0]):
    for r in range(0,integrals_ijrs.shape[1]):
        for i in range(0,two_electron_integrals.shape[0]):
            for j in range(0,two_electron_integrals.shape[1]):
                for k in range(0,two_electron_integrals.shape[2]):
                    integrals_ijrs[i,j,r,s]+=C_ao_mo[k,r]*integrals_ijks[i,j,k,s]

for s in range(0,integrals_iqrs.shape[0]):
    for r in range(0,integrals_iqrs.shape[1]):
        for q in range(0,integrals_iqrs.shape[2]):
            for i in range(0,two_electron_integrals.shape[0]):
                for j in range(0,two_electron_integrals.shape[1]):
                    integrals_iqrs[i,q,r,s]+=C_ao_mo[j,q]*integrals_ijrs[i,j,r,s]

for s in range(0,integrals_pqrs.shape[0]):
    for r in range(0,integrals_pqrs.shape[1]):
        for q in range(0,integrals_pqrs.shape[2]):
            for p in range(0,integrals_pqrs.shape[3]):
                for i in range(0,two_electron_integrals.shape[0]):
                    integrals_pqrs[p,q,r,s]+=C_ao_mo[i,p]*integrals_iqrs[i,q,r,s]
                    
    
#for s in range(0,integrals_pqrs.shape[0]):
 #   for r in range(0,integrals_pqrs.shape[1]):
  #      for q in range(0,integrals_pqrs.shape[2]):
   #         for p in range(0,integrals_pqrs.shape[3]):             
print(integrals_pqrs)



[[[[ 6.52472109e-01 -4.24139890e-16 -1.66519354e-01  2.86229374e-17]
   [-5.19549681e-16  4.55558973e-01 -4.76615275e-16  1.43457642e-01]
   [-1.66519354e-01 -5.03937170e-16  5.60686956e-01  3.20923843e-16]
   [ 3.29597460e-17  1.43457642e-01  3.47812057e-16  6.67302158e-01]]

  [[-5.06539255e-16  8.90697529e-02 -2.47198095e-16  8.00945407e-02]
   [ 8.90697529e-02 -4.57966998e-16  1.67063870e-02  1.23165367e-16]
   [-2.54136989e-16  1.67063870e-02  5.55111512e-17 -9.33134356e-02]
   [ 8.00945407e-02  4.77048956e-17 -9.33134356e-02 -9.22872889e-16]]

  [[-1.66519354e-01 -2.35922393e-16  1.14220829e-01 -8.56519716e-17]
   [-1.34007389e-16 -5.56499435e-02  2.48499138e-16 -7.75908237e-02]
   [ 1.14220829e-01  2.77989437e-16 -1.32706307e-01  4.97540377e-16]
   [-1.61979805e-16 -7.75908237e-02  4.79434201e-16 -2.03364455e-01]]

  [[ 9.80118764e-17  8.00945407e-02 -7.37257477e-17  1.31150506e-01]
   [ 8.00945407e-02  1.27393755e-16 -2.69543050e-02 -1.39645240e-16]
   [-9.36750677e-17 -2.69543

In [66]:
B = np.zeros((mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0]))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,B.shape[1]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,B.shape[3]):
                B[a,r,b,s]+=integrals_pqrs[r,s,a,b]-integrals_pqrs[r,s,b,a]
B

array([[[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]],


       [[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

      

In [67]:
A = np.zeros((mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0],mo_integrals.shape[0]))

for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,B.shape[1]):
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,B.shape[3]):
                A[a,r,b,s]+=mo_integrals[r,b,a,s]-mo_integrals[r,b,s,a]
                if a==b and r==s:
                    A[a,r,b,s]+= (F_diag_procedure[0][r] - F_diag_procedure[0][a])

#for i in range(0,A.shape[0]):
#    A[i,i,i,i]=A[i,i,i,i]+

In [ ]:
B = np.zeros((mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2),mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2)))

#mf = scf.RHF(mol).run()
#mo_ints = ao2mo.kernel(mol, mf.mo_coeff)
#mo_ints = ao2mo.restore(1, mo_ints, mol.nao_nr())
#mo_integrals = mo_ints

arindex = -1
for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,overlap_integrals.shape[0]):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,overlap_integrals.shape[0]):
                bsindex += 1
                B[arindex,bsindex]+=2*mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a]
               # lol[a,b] = 1
               # lol[r,s] = 2
                #print(r,s,a,b,mo_integrals[r,s,a,b]-mo_integrals[r,s,b,a],mo_integrals[r,s,a,b],mo_integrals[r,s,b,a])
#assert np.allclose(B, np.zeros(B.shape))
print(B)

In [ ]:
A = np.zeros((mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2),mol.nelectron//2*(overlap_integrals.shape[0]-mol.nelectron//2)))

arindex = -1
for a in range(0,mol.nelectron//2):
    for r in range(mol.nelectron//2,overlap_integrals.shape[0]):
        arindex += 1
        bsindex = -1
        for b in range(0,mol.nelectron//2):
            for s in range(mol.nelectron//2,overlap_integrals.shape[0]):
                bsindex += 1
                A[arindex,bsindex]+=2*mo_integrals[r,b,a,s]-mo_integrals[r,b,s,a]
                if a==b:
                    A[arindex,bsindex] += F_diag_procedure[0][r]
                if r==s:
                    A[arindex,bsindex] += - F_diag_procedure[0][a]
                    
print(A)